In [1]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import feature, exposure, color
from skimage.transform import pyramid_gaussian
from skimage.io import imread
import sklearn.svm
from sklearn.svm import LinearSVC
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from skimage import color
from PIL import Image 
import argparse
import cv2
import os
import glob

In [ ]:
# diretorio das imagens positivas
pos_im_path = r"./images/gadoP" 
# diretorio das imagens negativas
neg_im_path= r"./images/gadoN"

# le as imagens
pos_im_listing = os.listdir(pos_im_path)
neg_im_listing = os.listdir(neg_im_path)
num_pos_samples = len(pos_im_listing) 
num_neg_samples = len(neg_im_listing)
# print do numero de imagens em cada dataset
print(num_pos_samples)
print(num_neg_samples)

In [ ]:
def show_hog_images(img, hog_image):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 9), sharex=True, sharey=True)

    ax1.axis('off')
    ax1.imshow(img, cmap=plt.cm.gray)
    ax1.set_title('Input image')

    # Reescala o diagrama de gradientes para melhor visualização
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

    ax2.axis('off')
    ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
    ax2.set_title('Histogram of Oriented Gradients')
    plt.show()

In [ ]:
def ajust_img(img):
    img = cv2.resize(img,(64,64))
    img = cv2.medianBlur(img, 9)
    return img

In [ ]:
# Parametros do HoG
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)
threshold = .3

data= []
labels = []

def get_fd_hog_show(img):
    img = ajust_img(img)
    fd, hog = feature.hog(img, orientations, pixels_per_cell, cells_per_block,
                        block_norm='L2', visualize=True, feature_vector=True)
    show_hog_images(img, hog)
    
def get_fd_hog(img):
    img = ajust_img(img)
    fd = feature.hog(img, orientations, pixels_per_cell, cells_per_block,
                        block_norm='L2', feature_vector=True)# fd= feature descriptor
    return fd

# computar HoG para as todas imagens positivas
for file in pos_im_listing: 
    img = cv2.imread(pos_im_path + '/' + file) # abre o arquivo
    fd = get_fd_hog(img)
    data.append(fd)
    labels.append(1)
    
# mostra a ultima imagem positiva com o HoG ao lado
get_fd_hog_show(img)


# computar HoG para as todas imagens negativas
for file in neg_im_listing:
    img= cv2.imread(neg_im_path + '/' + file) # abre o arquivo
    fd = get_fd_hog(img)
    data.append(fd)
    labels.append(0)

# mostra a ultima imagem negativa com o HoG ao lado
get_fd_hog_show(img)
    
# codifca as labels, converte de string pra int
le = LabelEncoder()
labels = le.fit_transform(labels)



In [ ]:
# img.shape
img.size

In [ ]:
# Criacao do modelo
# usando 80% pra treino, e 20% pra teste
print(" Construindo modelo e seprandoo dataset ...")
(trainData, testData, trainLabels, testLabels) = train_test_split(
	np.array(data), labels, test_size=0.20, random_state=42)
# Treina SVM
print(" Treinando modelo SVM ...")
model = LinearSVC()
model.fit(trainData, trainLabels)
# Avalia o classificador
print(" Avaliando o classificador no teste ...")
predictions = model.predict(testData)
print(classification_report(testLabels, predictions))


# Save o modelo
joblib.dump(model, 'model_bovinos.npy')

In [ ]:
# Testa Modelo treinado

# Parametros HOG
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)
threshold = .3


def sliding_window(image, stepSize, windowSize):# image is the input, step size is the no.of pixels needed to skip and windowSize is the size of the actual window
    for y in range(0, image.shape[0], stepSize):# this line and the line below actually defines the sliding part and loops over the x and y coordinates
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y: y + windowSize[1], x:x + windowSize[0]])

# Salva o modelo treinado
model = joblib.load('model_bovinos.npy')

# Teste do treino com uma imagem
scale = 0
detections = []
img = cv2.imread("images/testP/135.png") # open the file

img = ajust_img(img)

(winW, winH)= (64,128)
windowSize=(winW,winH)
downscale=1.5
# Loop pra cada novo zoom na imagem
for resized in pyramid_gaussian(img, downscale=1.5):
    for (x,y,window) in sliding_window(resized, stepSize=10, windowSize=(winW,winH)):
        # se a window n tiver o msm tamanho da imagem, ignorar
        if window.shape[0] != winH or window.shape[1] !=winW:
            continue
        window=color.rgb2gray(window)
        fds, hog = feature.hog(window, orientations, pixels_per_cell, cells_per_block, block_norm='L2')  # extract HOG features from the window captured
        fds = fds.reshape(1, -1) # re shape the image to make a silouhette of hog
        pred = model.predict(fds) # use the SVM model to make a prediction on the HOG features extracted from the window
        
        if pred == 1:
            if model.decision_function(fds) > 0.6:  # set a threshold value for the SVM prediction i.e. only firm the predictions above probability of 0.6
                print("Detection:: Location -> ({}, {})".format(x, y))
                print("Scale ->  {} | Confidence Score {} \n".format(scale,model.decision_function(fds)))
                detections.append((int(x * (downscale**scale)), int(y * (downscale**scale)), model.decision_function(fds),
                                   int(windowSize[0]*(downscale**scale)), # create a list of all the predictions found
                                      int(windowSize[1]*(downscale**scale))))
    scale+=1
    
clone = resized.copy()
for (x_tl, y_tl, _, w, h) in detections:
    cv2.rectangle(img, (x_tl, y_tl), (x_tl + w, y_tl + h), (0, 0, 255), thickness = 2)
rects = np.array([[x, y, x + w, y + h] for (x, y, _, w, h) in detections]) # do nms on the detected bounding boxes
sc = [score[0] for (x, y, score, w, h) in detections]
print("detection confidence score: ", sc)


In [ ]:
from tensorflow import image

sc = np.array(sc)
pick = image.non_max_suppression(rects, scores=sc, max_output_size=1, iou_threshold = 0.3)

# cria a caixa verde em volta da detecção
for (xA, yA, xB, yB) in pick:
    cv2.rectangle(img, (xA, yA), (xB, yB), (0,255,0), 2)
cv2.imshow("Raw Detections after NMS", img)
